# Comparison of CCT- & CMC-learners and other approaches (simulations)

**On Alaa et al. (2023) synthetic data**

In [1]:
import pytensor

pytensor.config.optimizer = 'fast_compile'  # or 'None' for no optimizations
pytensor.config.exception_verbosity = 'high'

import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
from crepes_weighted import WrapRegressor


from tqdm.notebook import tqdm


sys.path.append('../..')
from src.cmc_metalearners.cmc_metalearners import CMC_S_Learner, CMC_T_Learner, CMC_X_Learner, CCT_Learner
from src.wcp.wcp import NaiveWCP, NestedWCP
from src.conformal_metalearners.CM_learner import CM_learner
from src.datasets.alaa_synthetic import (generate_data)
from src.benchmarks.bart import BART
from src.benchmarks.cmgp import CMGP
from src.benchmarks.cevae import CEVAE
from src.benchmarks.fccn import FCCN
from src.benchmarks.ganite.ganite import GANITE
from src.benchmarks.dklite import DKLITE
from src.benchmarks.diffpo.main_model import DiffPOITE
from src.benchmarks.noflite.noflite import NOFLITE
import torch

## Comparison Setups

In [2]:
get_save_data = True
NSim = 100
alpha = 0.1
learner = RandomForestRegressor
learner_name = "RF"
MC_samples = 100
normalized_conformal = True
if normalized_conformal:
    normalized_conformal_name = "normalized"
else:
    normalized_conformal_name = "nonnormalized"
max_min_y = True

setup_A = {"n":5000, "d": 10, "gamma":1, "alpha": 0.1, "nexps": 1}
setup_B = {"n":5000, "d": 10, "gamma":0, "alpha": 0.1, "nexps": 1}

In [3]:
for i, setup in enumerate([setup_A, setup_B]):
    if i == 0:
        continue
    setup_name = "A" if i == 0 else "B"
    for n in tqdm(range(NSim)):
    # for n in tqdm(range(NSim)):
        path_train = f"../../data/simulations/alaa/setup{setup_name}/simulations_{setup_name}_{str(n)}_train.csv"
        path_test = f"../../data/simulations/alaa/setup{setup_name}/simulations_{setup_name}_{str(n)}_test.csv"
        if get_save_data:
            ds_train = pd.read_csv(path_train)
            ds_test = pd.read_csv(path_test)
        else:
            ds = generate_data(**setup)
            ds_train, ds_test = train_test_split(ds[0], test_size=0.6, random_state=42)
            ds_train.to_csv(path_train)
            ds_test.to_csv(path_test)
        W_train = ds_train['T'].to_numpy()
        y_train = ds_train['Y'].to_numpy()
        y1_train = ds_train['Y1'].to_numpy()
        y0_train = ds_train['Y0'].to_numpy()
        X_train = ds_train[['X'+str(i) for i in range(1,setup_A["d"]+1)]].to_numpy()
        ps_train = ds_train['ps'].to_numpy()
        ite_train = y1_train - y0_train

        W_test = ds_test['T'].to_numpy()
        y_test = ds_test['Y'].to_numpy()
        y1_test = ds_test['Y1'].to_numpy()
        y0_test = ds_test['Y0'].to_numpy()
        X_test = ds_test[['X'+str(i) for i in range(1,setup_A["d"]+1)]].to_numpy()
        ps_test = ds_test['ps'].to_numpy()
        ite_test = y1_test - y0_test

        # # Initialize the learner
        # conformal_pseudo_MC_T_Learner = CMC_T_Learner(
        #     learner(),
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=True,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )
        # conformal_pseudo_MC_T_Learner.fit(X_train, y_train, W_train, ps_train)

        # conformal_MC_T_Learner = CMC_T_Learner(
        #     learner(),
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=False,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )
        # conformal_MC_T_Learner.fit(X_train, y_train, W_train, ps_train)

        # conformal_pseudo_MC_S_Learner = CMC_S_Learner(
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=True,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )
        # with warnings.catch_warnings():
        #     conformal_pseudo_MC_S_Learner.fit(X_train, y_train, W_train, ps_train)

        # conformal_MC_S_Learner = CMC_S_Learner(
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=False,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )

        # with warnings.catch_warnings():
        #     conformal_MC_S_Learner.fit(X_train, y_train, W_train, ps_train)


        # conformal_pseudo_MC_X_Learner = CMC_X_Learner(
        #     learner(),
        #     learner(),
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=True,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )
        # # Fit the learner
        # with warnings.catch_warnings():
        #     warnings.simplefilter("ignore")
        #     conformal_pseudo_MC_X_Learner.fit(X_train, y_train, W_train, ps_train)
        # conformal_MC_X_Learner = CMC_X_Learner(
        #     learner(),
        #     learner(),
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=False,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )

        # # Fit the learner
        # with warnings.catch_warnings():
        #     warnings.simplefilter("ignore")
        #     conformal_MC_X_Learner.fit(X_train, y_train, W_train, ps_train)

        (X_train_nuisance, X_train_cal,
            y_train_nuisance, y_train_cal,
            y0_train_nuisance, y0_train_cal,
            y1_train_nuisance, y1_train_cal,
            W_train_nuisance, W_train_cal,
            ite_train_nuisance, ite_train_cal) = train_test_split(
                X_train, y_train, y0_train, y1_train, W_train, ite_train, test_size=0.5, random_state=n
        )

        conformal_y0 = WrapRegressor(learner())
        conformal_y0.fit(X_train_nuisance, y0_train_nuisance)
        conformal_y0.calibrate(X_train_cal, y0_train_cal, cps=True)
        conformal_y1 = WrapRegressor(learner())
        conformal_y1.fit(X_train_nuisance, y1_train_nuisance)
        conformal_y1.calibrate(X_train_cal, y1_train_cal, cps=True)
        conformal_ite = WrapRegressor(learner())
        conformal_ite.fit(X_train_nuisance, ite_train_nuisance)
        conformal_ite.calibrate(X_train_cal, ite_train_cal, cps=True)

        df_eval = pd.DataFrame(columns=["approach",
                                        "rmse_y0", "rmse_y1", "rmse_ite",
                                        "coverage_y0", "coverage_y1", "coverage_ite",
                                        "efficiency_y0", "efficiency_y1", "efficiency_ite",
                                        "crps_y0", "crps_y1", "crps_ite",
                                        "ll_y0", "ll_y1", "ll_ite"])

        # CCT-learner
        print("Fit and evaluate CCT-learner ...")
        conformal_CT_learner = CCT_Learner(learner(), learner(), normalized_conformal=normalized_conformal)
        conformal_CT_learner.fit(X_train, y_train, W_train, p=ps_train)
        evaluate = conformal_CT_learner.evaluate(X_test, y0_test, y1_test, ps_test, alpha=alpha)
        evaluate["approach"] = "CTT-learner"
        df_eval = pd.concat([df_eval, pd.DataFrame(evaluate, index=[0])], ignore_index=True)

        # # BART
        print("Fit and evaluate BART ...")
        bart = BART()
        bart.fit(X_train, y_train, W_train)
        evaluate = bart.evaluate(X_test, y0_test, y1_test, alpha=alpha)
        evaluate["approach"] = "BART"
        df_eval = pd.concat([df_eval, pd.DataFrame(evaluate, index=[0])], ignore_index=True)

        # CMGP
        print("Fit and evaluate CMGP ...")
        cmgp = CMGP(X=X_train, Treatments=W_train, Y=y_train)
        evaluate = cmgp.evaluate(X_test, y0_test, y1_test, alpha=alpha)
        evaluate["approach"] = "CMGP"
        df_eval = pd.concat([df_eval, pd.DataFrame(evaluate, index=[0])], ignore_index=True)

        # CEVAE
        print("Fit and evaluate CEVAE ...")
        dim_bin = 0
        dim_cont = X_train.shape[1]
        cevae = CEVAE(dim_bin=dim_bin, dim_cont=dim_cont)
        cevae.fit(X=X_train, Y=y_train, W=W_train)
        evaluate = cevae.evaluate(X_test, y0_test, y1_test, alpha=alpha)
        evaluate["approach"] = "CEVAE"
        df_eval = pd.concat([df_eval, pd.DataFrame(evaluate, index=[0])], ignore_index=True)

        # FCCN
        print("Fit and evaluate FCCN ...")
        fccn = FCCN(input_size=X_train.shape[1])
        fccn.train(X_train, y_train, W_train)
        evaluate = fccn.evaluate(X_test, y0_test, y1_test, alpha=alpha)
        evaluate["approach"] = "FCCN"
        df_eval = pd.concat([df_eval, pd.DataFrame(evaluate, index=[0])], ignore_index=True)

        # GANITE
        print("Fit and evaluate GANITE ...")
        ganite = ganite_params = {'h_dim': 30,                          # hidden dimensions
                                 'batch_size': 64,                     # the number of samples in each batch
                                 'iterations': 10000,                  # the number of iterations for training
                                 'alpha': 2.,
                                 'beta': 5.,                           # hyper-parameter to adjust the loss importance
                                 'input_size': X_train.shape[1],       # the number of features
                                 }
        ganite = GANITE(**ganite_params)
        ganite.fit(X_train, y_train, W_train)
        evaluate = ganite.evaluate(X_test, y0_test, y1_test, alpha=alpha)
        evaluate["approach"] = "GANITE"
        df_eval = pd.concat([df_eval, pd.DataFrame(evaluate, index=[0])], ignore_index=True)

        # DKLITE
        print("Fit and evaluate DKLITE ...")
        dklite = DKLITE(input_dim=X_train.shape[1], output_dim=1)
        dklite.fit(X_train, y_train, W_train)
        evaluate = dklite.evaluate(X_test, y0_test, y1_test, alpha=alpha)
        evaluate["approach"] = "DKLITE"
        df_eval = pd.concat([df_eval, pd.DataFrame(evaluate, index=[0])], ignore_index=True)

        # diffpo
        print("Fit and evaluate diffpo ...")
        config = {
            "train": {
                "epochs": 500,
                "batch_size": 256,
                "lr": 0.0005,
                "valid_epoch_interval": 50
            },
            "diffusion": {
                "layers": 4,
                "channels": 64,
                "f_dim": 180,
                "cond_dim": X_train.shape[1] + 1, # conditional variable dimension
                "hidden_dim": 128,
                "side_dim": 33,
                "nheads": 2,
                "diffusion_embedding_dim": 128,
                "beta_start": 0.0001,
                "beta_end": 0.5,
                "num_steps": 100,
                "schedule": "quad",
                "mixed": False
            },
            "model": {
                "is_unconditional": 0,
                "timeemb": 32,
                "featureemb": 16,
                "target_strategy": "random",
                "mixed": False
            },
        }
        device = "cuda" if torch.cuda.is_available() else "cpu"
        diffpo = DiffPOITE(config=config,
                           device=device)
        diffpo.fit(X=X_train, Y0=y0_train, Y1=y1_train, W=W_train, ps=ps_train)
        evaluate = diffpo.evaluate(X_test, y0_test, y1_test, W=W_test, ps=ps_test, alpha=alpha)
        evaluate["approach"] = "DiffPO"
        df_eval = pd.concat([df_eval, pd.DataFrame(evaluate, index=[0])], ignore_index=True)

        # NOFLITE
        print("Fit and evaluate NOFLITE ...")
        params = {
            'input_size': X_train.shape[1],
            'lr': 5e-4,
            'lambda_l1': 1e-3,
            'lambda_l2': 5e-3,
            'batch_size': 128,
            'noise_reg_x': 1e-0,
            'noise_reg_y': 5e-1,
            'hidden_neurons_encoder': 25,
            'hidden_layers_balancer': 3,
            'hidden_layers_encoder': 0,
            'hidden_layers_prior': 2,
            'hidden_neurons_trans': 4,
            'hidden_neurons_cond': 16,
            'hidden_layers_cond': 2,
            'dense': False,
            'n_flows': 1,
            'datapoint_num': 8,
            'resid_layers': 1,
            'max_steps': 10000,
            'flow_type': "SigmoidX",
            'metalearner': "T",
            'lambda_mmd': 0.1,
            'n_samples': 500,
            'trunc_prob': 0.01,
            'bin_outcome': False,
            'iterations': 1,
            'visualize': False,
        }
        noflite = NOFLITE(params=params)
        noflite.fit(X_train, y_train, W_train)
        evaluate = noflite.evaluate(X_test, y0_test, y1_test, W=W_test, alpha=alpha)
        evaluate["approach"] = "NOFLITE"
        df_eval = pd.concat([df_eval, pd.DataFrame(evaluate, index=[0])], ignore_index=True)

        if max_min_y:
            df_eval.to_csv(f"../../results/outputs/alaa/setup{setup_name}/eval_dist/simulations_{setup_name}_{str(n)}_{learner_name}_{normalized_conformal_name}_max_min_y_eval.csv", index=False)
        else:
            df_eval.to_csv(f"../../results/outputs/alaa/setup{setup_name}/eval_dist/simulations_{setup_name}_{str(n)}_{learner_name}_{normalized_conformal_name}_eval.csv", index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

Fit and evaluate CCT-learner ...
Fit and evaluate BART ...


Multiprocess sampling (4 chains in 4 jobs)
PGBART: [mu]


Output()

Sampling 4 chains for 500 tune and 500 draw iterations (2_000 + 2_000 draws total) took 18 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Sampling: [mu, y]


...

Fit and evaluate DKLITE ...


  5%|▌         | 51/1000 [00:02<00:55, 17.17it/s]


Fit and evaluate diffpo ...


  0%|          | 0/7 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [4]:
n

1

## Evaluating Probability Calibrations

In [6]:
setup_A = {"n":3001, "d": 10, "gamma":1, "alpha": 0.1, "nexps": 1}
setup_B = {"n":3001, "d": 10, "gamma":0, "alpha": 0.1, "nexps": 1}

learner = RandomForestRegressor
learner_name = "RF"
MC_samples = 100
normalized_conformal = True
if normalized_conformal:
    normalized_conformal_name = "normalized"
else:
    normalized_conformal_name = "nonnormalized"
max_min_y = True

PROB_NSIM = 1000

In [ ]:
for i, setup in enumerate([setup_A, setup_B]):
    list_p_values_pseudo_MC_T = []
    list_p_values_MC_T = []
    list_p_values_CT = []
    list_p_values_y0 = []
    list_p_values_y1 = []
    list_p_values_oracle = []
    list_p_values_pseudo_MC_T_unweighted = []
    list_p_values_MC_T_unweighted = []
    list_p_values_CT_unweighted = []
    list_p_values_y0_unweighted = []
    list_p_values_y1_unweighted = []
    setup_name = "A" if i == 0 else "B"
    for n in tqdm(range(PROB_NSIM)):
        ds = generate_data(**setup)
        ds_train, ds_test = train_test_split(ds[0], test_size=1)
        W_train = ds_train['T'].to_numpy()
        y_train = ds_train['Y'].to_numpy()
        y1_train = ds_train['Y1'].to_numpy()
        y0_train = ds_train['Y0'].to_numpy()
        X_train = ds_train[['X'+str(i) for i in range(1,setup_A["d"]+1)]].to_numpy()
        ps_train = ds_train['ps'].to_numpy()
        ite_train = y1_train - y0_train

        W_test = ds_test['T'].to_numpy()
        y_test = ds_test['Y'].to_numpy()
        y1_test = ds_test['Y1'].to_numpy()
        y0_test = ds_test['Y0'].to_numpy()
        X_test = ds_test[['X'+str(i) for i in range(1,setup_A["d"]+1)]].to_numpy()
        ps_test = ds_test['ps'].to_numpy()
        ite_test = y1_test - y0_test

        ## Initialize and fit the learners (weighted)
        conformal_pseudo_MC_T_Learner = CMC_T_Learner(
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=True,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        conformal_pseudo_MC_T_Learner.fit(X_train, y_train, W_train, ps_train)

        conformal_MC_T_Learner = CMC_T_Learner(
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=False,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        conformal_MC_T_Learner.fit(X_train, y_train, W_train, ps_train)

        conformal_CT_learner = CCT_Learner(learner(), learner(), normalized_conformal=normalized_conformal)
        conformal_CT_learner.fit(X_train, y_train, W_train, p=ps_train)
        (X_train_nuisance, X_train_cal,
            y_train_nuisance, y_train_cal,
            y0_train_nuisance, y0_train_cal,
            y1_train_nuisance, y1_train_cal,
            W_train_nuisance, W_train_cal,
            ite_train_nuisance, ite_train_cal) = train_test_split(
                X_train, y_train, y0_train, y1_train, W_train, ite_train, test_size=0.5, random_state=n
        )
        conformal_ite_oracle = WrapRegressor(learner())
        conformal_ite_oracle.fit(X_train_nuisance, ite_train_nuisance)
        conformal_ite_oracle.calibrate(X_train_cal, ite_train_cal, cps=True)
        # p-values
        list_p_values_pseudo_MC_T.append(conformal_pseudo_MC_T_Learner.predict_p_value(X_test, ite_test))
        list_p_values_MC_T.append(conformal_MC_T_Learner.predict_p_value(X_test, ite_test))
        list_p_values_CT.append(conformal_CT_learner.predict_p_value(X_test, ite_test, p=ps_test))
        list_p_values_y0.append(conformal_CT_learner.predict_p_value_y0(X_test, y0_test, p=ps_test))
        list_p_values_y1.append(conformal_CT_learner.predict_p_value_y1(X_test, y1_test, p=ps_test))
        list_p_values_oracle.append(conformal_ite_oracle.predict_cps(X_test, y=ite_test))

        ## Initialize and fit the learners (unweighted)
        conformal_pseudo_MC_T_Learner = CMC_T_Learner(
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=True,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        conformal_pseudo_MC_T_Learner.fit(X_train, y_train, W_train)

        conformal_MC_T_Learner = CMC_T_Learner(
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=False,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        conformal_MC_T_Learner.fit(X_train, y_train, W_train)

        conformal_CT_learner = CCT_Learner(learner(), learner(), normalized_conformal=normalized_conformal)
        conformal_CT_learner.fit(X_train, y_train, W_train)
        # p-values
        list_p_values_pseudo_MC_T_unweighted.append(conformal_pseudo_MC_T_Learner.predict_p_value(X_test, ite_test))
        list_p_values_MC_T_unweighted.append(conformal_MC_T_Learner.predict_p_value(X_test, ite_test))
        list_p_values_CT_unweighted.append(conformal_CT_learner.predict_p_value(X_test, ite_test))
        list_p_values_y0_unweighted.append(conformal_CT_learner.predict_p_value_y0(X_test, y0_test))
        list_p_values_y1_unweighted.append(conformal_CT_learner.predict_p_value_y1(X_test, y1_test))



    dict_p_values = {
            "pseudo_MC_T": np.concatenate(list_p_values_pseudo_MC_T),
            "MC_T": np.concatenate(list_p_values_MC_T),
            "CT": np.concatenate(list_p_values_CT),
            "y0": np.concatenate(list_p_values_y0),
            "y1": np.concatenate(list_p_values_y1),
            "oracle": np.concatenate(list_p_values_oracle),
            "pseudo_MC_T_unweighted": np.concatenate(list_p_values_pseudo_MC_T_unweighted),
            "MC_T_unweighted": np.concatenate(list_p_values_MC_T_unweighted),
            "CT_unweighted": np.concatenate(list_p_values_CT_unweighted),
            "y0_unweighted": np.concatenate(list_p_values_y0_unweighted),
            "y1_unweighted": np.concatenate(list_p_values_y1_unweighted),
    }
    df_p_values = pd.DataFrame(dict_p_values)
    if max_min_y:
        df_p_values.to_csv(f"../../results/outputs/alaa/setup{setup_name}/p_values/simulations_{setup_name}_{learner_name}_{normalized_conformal_name}_max_min_y_p_values.csv", index=False)
    else:
        df_p_values.to_csv(f"../../results/outputs/alaa/setup{setup_name}/p_values/simulations_{setup_name}_{learner_name}_{normalized_conformal_name}_p_values.csv", index=False)